In [ ]:
!pip install -r ../src/train/requirements.txt --quiet
!pip install kafka-python --quiet

Restart the Jupyter kernel from the menu after installing pip packages.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import dvc.api
import s3fs
import os

resource_url = dvc.api.get_url(
    path='creditcard-produce.csv',
    repo=os.environ['DATA_REPO'],
    rev='v1.0')

print("Data is from this S3 bucket:\n{}".format(resource_url))

# Pandas doens't support endpoint_url
# https://github.com/pandas-dev/pandas/pull/29050
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': os.environ['S3_ENDPOINT_URL']})
df = pd.read_csv(fs.open(resource_url))

In [ ]:
df.head()

In [ ]:
topic='{}-prod'.format(os.environ['WORKSHOP_USER_ID'])
print('Kafka topic: {}'.format(topic))

In [ ]:
bootstrap_servers='kafka-kafka-bootstrap.labs-kafka.svc:9092'

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
producer = KafkaProducer(value_serializer=lambda x: x.encode('utf-8'), \
                         bootstrap_servers=bootstrap_servers)
print('Is connected? {}'.format(producer.bootstrap_connected()))

In [ ]:
def send(row):
    producer.send(topic,                              
                  value=str(row.values.tolist()))
    print('Customer ID: {} Class: {}'.format(row['Customer_ID'], row['Class']))

In [ ]:
_ = df.apply(send, axis=1)